In [ ]:
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


def readPdf(list_file):
    """
    Fonction qui transforme des pdf en texte :

    Args:
        file (list): liste des fichiers

    Returns:
        text : contenu du fichier
    """
    list_texts = []
    for file in list_file:
        reader = PdfReader(file)
        nb_pages = len(reader.pages)
        for i in range(nb_pages):
            page = reader.pages[i]
            text = page.extract_text()
            list_texts.append(text)
    return list_texts

def chunks(s, k):
    """
    Fonction qui prend du texte et le découpe en chunk de k taille

    Args:
        s (text): texte à découper
        k (int): découpage en k chunks

    Returns:
        list_texts : liste de liste texte découpé
    """
    return [s[i:i+k] for i in range(0, len(s), k)]


document = readPdf(["pdf-exemple.pdf","sample.pdf","rugby.pdf","mes-fiches-animaux-de-la-ferme.pdf","vaches.pdf"])
#document = readPdf([])
print(document)
docs = [Document(page_content=page_text) for page_text  in document]
splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap=50,separators=["\n\n","\n",".","!","?"])

splits_docs = splitter.split_documents(docs)
print(splits_docs)


chunks = [doc.page_content for doc in splits_docs]

print(len(chunks))
#On embedding (transférer les phrases par des vecteurs)
modelEmbedding = SentenceTransformer("all-MiniLM-L6-v2")



["Fichier PDF d'exemple \nLe Portable Document Format (qui se traduit de l'anglais en « format de document \nportable »), généralement abrégé PDF, est un format de fichier informatique créé par \nAdobe  Systems.  C'est  un  format  ouvert  dont  les  spécifications  sont  publiques  et \nutilisables librement (certains éléments sont à disposition sur le site Adobe). Il est \ndérivé du format PostScript et contient des données au format XML.\nLe format PDF est un format de fichier qui préserve les polices, les images, les objets \ngraphiques et la mise en forme de tout document source, quelles que soient l'application \net la plate-forme utilisées pour le créer. Les fichiers PDF peuvent être créés avec des \noptions personnalisées, tant aux niveaux de la compression des images et des textes, \nde la qualité d'impression du fichier, ainsi que du verrouillage (interdiction d'impression, \nde modification...).\nLe format PDF n'est pas un format statique mais un format interactif. Il est en

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

[[-5.98347113e-02  3.84279452e-02 -9.98053476e-02 ...  2.06796154e-02
   9.51356813e-02  5.70650660e-02]
 [-5.53533621e-02  9.79981050e-02 -8.81400406e-02 ...  2.23702844e-02
   9.63604897e-02  1.46429420e-05]
 [-7.14292675e-02  1.00834884e-01 -9.87062827e-02 ... -2.36885510e-02
   5.09376079e-02 -1.26135852e-02]
 ...
 [-4.17675115e-02 -5.25434166e-02 -3.88462506e-02 ... -1.36453900e-02
   9.46532413e-02 -4.59779762e-02]
 [ 5.79336146e-03 -3.12568136e-02 -3.89261767e-02 ...  3.94407213e-02
   1.00369893e-01 -6.73145354e-02]
 [ 4.68622483e-02 -1.05003633e-01 -1.34475809e-02 ... -2.22385582e-03
   8.14229101e-02 -2.08854508e-02]]


In [ ]:
from sentence_transformers import SentenceTransformer, util

user_text = input("Entrez votre phrase : ")
modelEmbedding = SentenceTransformer("all-MiniLM-L6-v2")
embedding_user = modelEmbedding.encode(user_text,show_progress_bar=True)
print(embedding_user)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[-2.90519390e-02  2.08551921e-02 -4.68506105e-03 -3.35217541e-04
  3.12589593e-02  1.90376714e-02  4.66589779e-02  3.12959217e-02
  7.58860484e-02  2.29434371e-02  6.66845292e-02 -1.03296287e-01
  5.90442531e-02 -4.61206585e-02 -6.10138774e-02 -5.22394925e-02
 -2.85218848e-04  5.05903773e-02  3.00416779e-02  3.97553481e-02
  4.75539500e-03 -5.95272593e-02 -4.49362583e-02  1.08072966e-01
 -1.43152531e-02  3.51818576e-02  4.39531989e-02 -3.21988314e-02
  4.21464862e-03 -1.05649037e-02 -1.93663482e-02  7.67020360e-02
  1.62408594e-02 -5.75940423e-02  1.28788892e-02  7.24878581e-03
  6.75900877e-02 -7.76548386e-02  9.38820839e-03  6.14480115e-03
 -5.02672344e-02 -2.33583488e-02  1.18732583e-02 -5.52151203e-02
  8.21704138e-03  7.14763105e-02  3.68511267e-02  5.99899366e-02
 -3.71507294e-02 -1.11812726e-02 -3.43575627e-02  6.20031096e-02
  8.29394087e-02  3.77615243e-02 -3.13737169e-02 -8.30258708e-03
  4.27409895e-02 -3.88920531e-02  2.22160928e-02 -2.25001033e-02
  2.67670024e-02 -1.18233

In [16]:
import math
import faiss
import numpy as np
import os 

#On utilise la recherche sémantique pour retrouver les vecteurs les plus proches (donc les plus probables)
# Création de l'index 

if os.path.exists("embedding.npy") and os.path.exists("query.npy"):
    embedding = np.load("embedding.npy")
    query = np.load("query.npy")
else:
    embedding = modelEmbedding.encode(chunks, show_progress_bar=True, convert_to_numpy=True).astype("float32")
    query = modelEmbedding.encode([user_text], show_progress_bar=True,convert_to_numpy=True).astype("float32")
    np.save("embedding.npy", embedding)
    np.save("query.npy",query)



dimension = embedding.shape[1]
index = faiss.IndexFlatL2(dimension)  # 32 = nombre de voisins (standard)
output_text = []

#on ajoute les embeddings : 
index.add(embedding)
print(f"Question de l'utilisateur : {user_text}\n")
distances, indices = index.search(query, k=5)
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. Score = {distances[0][i]:.2f} | {chunks[idx]}")
    output_text.append(chunks[idx])

Question de l'utilisateur : Comment est cultivé le maïs ?

1. Score = 0.86 | Figure 9 : Betteraves fourragères 
 
Les betteraves fourragères se cultivent d’une manière générale comme les sucrières. Elles 
sont cependant moins exigeantes en terme de qualité de sol, de sorte qu’on peut les cultive r 
partout, y compris en Ardenne . Par rapport au maïs, il s’agit d’une culture demandant
2. Score = 0.88 | caillebotis ou qui s’étalent comme de l’eau. 
Il s’agit des matières fécales d’un animal 
très malade.
3. Score = 0.90 | ma viande.
J'adore
barboter 
dans l'eau.
Je suis une volaille.
Un oiseau aquatique
de la famille des
anatidés.
4. Score = 0.97 | individualisée et la ration par lot.
5. Score = 0.99 | Je suis une 
 volaille. Un
oiseau de l'ordre
des gallinacés.


In [ ]:
DEFAULT_RESPONSE = "Désolé, les fichiers ne permettent pas de répondre à vos questions."

prompt_template = """
Contexte : Voici une liste d'extraits de document de référence, sélectionnés comme étant les plus pertinents selon la question. Utilise uniquement ces passages pour répondre en reformulant pour avoir une phrase lexicalement correcte.
Document fournis : """ + str(output_text) +  """\n
Consignes : \n
- Ne génère ta réponse qu'en te basant strictement sur le contenu des documents ci-dessus.
- Si les documents ne permettent pas de répondre clairement à la question, dis simplement : 
"Je ne peux pas répondre à cette question avec les documents fournis."
Question :
"""+ user_text  +"""
"""

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoModelForSeq2SeqLM
import time
from datetime import datetime
from csv import writer

t0 = time.time()
#model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model_name = "google/flan-t5-large"
#
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


if len(output_text) > 0 and len(document) > 0:
    #On remplace les mots par des vecteurs : 
    inputs = tokenizer(prompt_template,return_tensors="pt")
    print("Longueur token du prompt : ",tokenizer['input_ids'].shape[1])
    eos_token_id = tokenizer.eos_token_id  # Utilise le token EOS pour arrêter la génération

    # Permet de faire répondre le modèle sur les vecteurs que l'on vient de créer
    outputs = model.generate(**inputs, temperature=0.1, max_new_tokens=200, eos_token_id=eos_token_id)

    #On décode la réponse renvoyée par le modèle : 
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    print(response)
    
    #Pour les stats :
    result_response = "Réponse correcte"
elif len(document) == 0:
    prompt_template = """
    Répond à la question :""" + user_text
    
    #On remplace les mots par des vecteurs : 
    inputs = tokenizer(prompt_template,return_tensors="pt")
    print("Longueur token du prompt : ",tokenizer['input_ids'].shape[1])
    eos_token_id = tokenizer.eos_token_id  # Utilise le token EOS pour arrêter la génération

    # Permet de faire répondre le modèle sur les vecteurs que l'on vient de créer
    outputs = model.generate(**inputs, temperature=0.1, max_new_tokens=500, eos_token_id=eos_token_id)

    #On décode la réponse renvoyée par le modèle : 
    response = tokenizer.decode(outputs[0],skip_special_tokens=True) + "\n⚠️ CETTE REPONSE A ETE FOURNI SANS DOCUMENT ANNEXE, ELLE EST DONC GENERALE ⚠️"
    print(response)
    
    #Pour les stats :
    result_response = "Réponse hors-sujet sans fichier"
else:
    #Si il n'y a aucune réponse assez similaire (seuil 0.6)
    response = DEFAULT_RESPONSE
    result_response = "Hors-sujet"
t1 = time.time()
total_time = round(t1-t0,5)

#Inscription de la date : 
date_actuelle = datetime.now().strftime("%d/%m, %H:%M:%S")
# Inscription dans le fichier txt : 
with open("data_saved_v3.txt", "a", newline="", encoding="utf-8") as fichier:
    fichier.write("-------------------------------------------------------------------\n")
    fichier.write("Test du : " + date_actuelle + " avec modèle : " + model_name + "\n")
    fichier.write("-------------------------------------------------------------------\n")
    fichier.write("Temps d'éxecution : " + str(total_time) + "\n")
    fichier.write("Question posée : " + user_text + "\n")
    fichier.write("Réponse apportée : " + response + "\n")
    fichier.write("===================================================================\n\n")
fichier.close()



#Pareil mais en csv : 
dictio_csv = [model_name,date_actuelle,total_time,result_response]
with open("data_saved_v3.csv","a",encoding="utf-8",newline="") as fichier:
    w = writer(fichier)
    w.writerow(dictio_csv)
    fichier.close()




In [ ]:
#Inscription de la date : 
date_actuelle = datetime.now().strftime("%d/%m, %H:%M:%S")
# Inscription dans le fichier txt : 
with open("data_saved_v2.txt", "a", newline="", encoding="utf-8") as fichier:
    fichier.write("-------------------------------------------------------------------\n")
    fichier.write("Test du : " + date_actuelle + " avec modèle : " + model_name + "\n")
    fichier.write("-------------------------------------------------------------------\n")
    fichier.write("Temps d'éxecution : " + str(total_time) + "\n")
    fichier.write("Question posée : " + user_text + "\n")
    fichier.write("Réponse apportée : " + response + "\n")
    fichier.write("===================================================================\n\n")
fichier.close()



#Pareil mais en csv : 
dictio_csv = [model_name,date_actuelle,total_time,result_response]
with open("data_saved.csv","a",encoding="utf-8",newline="") as fichier:
    w = writer(fichier)
    w.writerow(dictio_csv)
    fichier.close()
